# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [3]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-cou

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [4]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [5]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [6]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
htt

In [7]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [8]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'project page', 'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'project page', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'company page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'patent',
   'url': 'https://patents.google.com/patent/US20210049536A1/'},
  {'type': 'linkedin profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'twitter profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'facebook profile',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

In [9]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [10]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
Found 4 relevant links


{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'company page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'professional profile',
   'url': 'https://www.linkedin.com/in/eddonner/'}]}

In [11]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 16 relevant links


{'links': [{'type': 'company homepage', 'url': 'https://huggingface.co/'},
  {'type': 'models page', 'url': 'https://huggingface.co/models'},
  {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'},
  {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'},
  {'type': 'documentation page', 'url': 'https://huggingface.co/docs'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'endpoints product page',
   'url': 'https://endpoints.huggingface.co'},
  {'type': 'community forum', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Zhihu page', 'url': 'https://www

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [13]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [14]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 12 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
Tongyi-MAI/Z-Image-Turbo
Updated
5 days ago
•
187k
•
2.23k
deepseek-ai/DeepSeek-V3.2
Updated
6 days ago
•
25.5k
•
764
deepseek-ai/DeepSeek-V3.2-Speciale
Updated
6 days ago
•
6.28k
•
529
microsoft/VibeVoice-Realtime-0.5B
Updated
about 5 hours ago
•
27.2k
•
406
alibaba-pai/Z-Image-Turbo-Fun-Controlnet-Union
Updated
5 days ago
•
260
Browse 1M+ models
Spaces
Running
on
Zero
MCP
Featured
1.17k
Z Image Turbo
🏃
1.17k
Generate images from text prompts
Running
on
Zero
324
Z Image Turbo
🖼
324
Generate stunning images from text prompts
Running


In [15]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [16]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [17]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 11 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nTongyi-MAI/Z-Image-Turbo\nUpdated\n5 days ago\n•\n187k\n•\n2.23k\ndeepseek-ai/DeepSeek-V3.2\nUpdated\n6 days ago\n•\n25.5k\n•\n764\ndeepseek-ai/DeepSeek-V3.2-Speciale\nUpdated\n6 days ago\n•\n6.28k\n•\n529\nmicrosoft/VibeVoice-Realtime-0.5B\nUpdated\nabout 5 hours ago\n•\n27.2k\n•\n406\nalibaba-pai/Z-Image-Turbo-Fun-Controlnet-Union\nUpdated\n5 days ago\n•\n260\nBrowse 1M+ models\nSpaces\nR

In [18]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [19]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 12 relevant links


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


# Hugging Face - The AI Community Building the Future

---

## About Hugging Face

Hugging Face is a premier AI platform and community where the global machine learning (ML) ecosystem collaborates to create models, datasets, and innovative applications. Known as "The Home of Machine Learning," Hugging Face empowers developers, researchers, and enterprises to accelerate ML development and deploy state-of-the-art AI solutions across multiple modalities including text, images, video, audio, and 3D.

---

## Platform Offerings

### Models
- Explore and contribute to over **1 million ML models**, including trending and cutting-edge projects such as Tongyi-MAI/Z-Image-Turbo and microsoft/VibeVoice-Realtime.
- Hosts models for varied AI use cases from image generation to real-time voice applications.

### Datasets
- Access and share from a vast collection of **250,000+ datasets** curated by the community, including datasets from industry leaders like Nvidia and Anthropic.
- Supports seamless discovery and collaboration for diverse AI training needs.

### Spaces
- Run and showcase AI applications easily in an interactive environment.
- Features thousands of live Spaces generating images, videos, and detailed evaluations which demonstrate the practical use of AI models.

### Community & Collaboration
- A vibrant and inclusive community that fosters sharing, learning, and building portfolios.
- Users can showcase their ML work globally, helping build influence and credibility.

### Enterprise Solutions
- Offers scalable **Team & Enterprise plans** with robust security, access controls, and dedicated support.
- Features include Single Sign-On (SSO), data region selection, audit logs, and flexible contracts to suit organizational needs.
- Pricing starts from $20/user/month for teams, with custom solutions for enterprises.

---

## Why Choose Hugging Face?

- **Open Source Stack:** Move faster by leveraging Hugging Face’s open-source ecosystem that supports all ML modalities.
- **Comprehensive Collaboration:** Host unlimited public content and collaborate effortlessly on models, datasets, and applications.
- **Accelerate ML Productivity:** Whether you are an individual developer or a large organization, Hugging Face has tools and services to speed up model development and deployment.
- **Community Powered:** Engage with millions of AI practitioners and researchers worldwide, sharing knowledge and driving innovation.
- **Enterprise Ready:** Built with enterprise-grade features ensuring security, compliance, and scalability.

---

## Company Culture

Hugging Face thrives on a community-first philosophy, nurturing openness, inclusivity, and collaboration. The company embraces innovation and continuous learning, encouraging contributors from all backgrounds to share knowledge freely and grow their careers in AI. The culture is driven by passion for open source technology and a commitment to advancing the future of artificial intelligence together.

---

## Careers & Opportunities

Hugging Face welcomes talented individuals passionate about AI and open source to join their team. They offer dynamic career paths in:

- Machine Learning Research & Engineering
- Software Development
- Community & Developer Relations
- Enterprise Solutions & Support
- Product Management

Contributors and employees thrive in an environment that values creativity, collaboration, and continuous impact on the AI landscape. Visit Hugging Face’s website to explore current job openings and become part of the AI revolution.

---

## Join Hugging Face Today

Whether you are a developer, researcher, enterprise, or an enthusiast, Hugging Face provides all the tools and community support you need to build and scale AI projects that shape the future.

- Explore AI Apps and Models
- Host and share your own projects
- Collaborate with a thriving AI community
- Scale production with enterprise-grade solutions

**Sign Up now** at [huggingface.co](https://huggingface.co) and be part of the AI community building tomorrow’s technology today.

---

*Hugging Face – Where Machine Learning Meets Community.*

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [20]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [21]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 4 relevant links


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is the vibrant, global AI and machine learning community building the future of open and ethical artificial intelligence. At its core, Hugging Face is a **collaboration platform** where ML engineers, scientists, and enthusiasts can share, discover, and experiment with a rich ecosystem of models, datasets, and applications.

The Hugging Face Hub is the central place for over **1 million machine learning models**, **250,000+ datasets**, and **400,000+ AI applications** spanning all modalities — text, image, video, audio, and 3D. This open platform fosters collaboration, accelerates innovation, and empowers users to build their AI portfolios and showcase their work worldwide.

---

## What We Offer

- **Models**: An extensive library of pretrained machine learning models, regularly updated and widely used for a variety of tasks.
- **Datasets**: Access to hundreds of thousands of open datasets, including those for autonomous vehicles, natural language processing, and more.
- **Spaces**: Easy-to-use environments where developers run and share AI apps, enabling seamless experimentations like image and video generation from text prompts.
- **Enterprise Solutions**: Advanced, scalable AI tools tailored for organizations needing enterprise-grade security, fine-grained access controls, dedicated support, and flexible compute options.
- **Community & Collaboration**: A thriving community engaged in continual development and ethical AI discussion, supported by forums, documentation, and open-source tools.

---

## Enterprise & Team Features

For organizations seeking to adopt or scale AI initiatives, Hugging Face offers:

- Flexible subscription plans starting at $20/user/month (Team plan) and custom contracts for Enterprises.
- **Single Sign-On (SSO)** for secure access integrating with existing identity providers.
- Granular **access controls**, **audit logs**, and **resource groups** for tight management of data and repositories.
- Enhanced storage options with private datasets and additional private storage capacity.
- Usage analytics dashboard and token management for organization-wide governance.
- Advanced compute power options including **ZeroGPU**, with increased quotas for enterprise members.

---

## Company Culture

Hugging Face thrives as a beacon of **open science**, **transparency**, and **collaboration**, deeply rooted in the principles of ethical AI development. The company nurtures innovation by empowering individuals and teams worldwide to contribute, learn, and lead in AI technology.

The community-first approach supports learning at all levels — from novices building AI portfolios to researchers pushing the edge of machine learning science.

---

## Who Uses Hugging Face?

- AI researchers and machine learning engineers
- Data scientists and developers building AI-powered products
- Enterprises looking to integrate secure and scalable AI solutions
- Educators and learners seeking to explore real-world applications of AI
- Innovators and creators harnessing AI for creative and scientific work

---

## Careers at Hugging Face

Join a company at the forefront of AI innovation! Hugging Face offers careers in research, engineering, product development, and community engagement. Working here means being part of a talented, passionate team committed to ethically advancing AI. 

Visit the [Hugging Face Jobs page](https://huggingface.co/jobs) to explore current openings and discover how you can contribute to the AI revolution.

---

## Get Started

- Explore models and datasets at [huggingface.co](https://huggingface.co)
- Browse AI applications and Spaces to see innovative demos.
- Sign up to build your portfolio and join the future of AI collaboration.

---

### Brand Colors

- Yellow: #FFD21E
- Orange: #FF9D00
- Gray: #6B7280

---

**Hugging Face**  
*The AI community building the future.*

---

For more information, visit: [https://huggingface.co](https://huggingface.co)  
Connect on [GitHub](https://github.com/huggingface), [Twitter](https://twitter.com/huggingface), [LinkedIn](https://linkedin.com/company/huggingface), and join the conversation on [Discord](https://discord.gg/huggingface).

In [22]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 4 relevant links


# Hugging Face Brochure

## About Hugging Face
Hugging Face is a vibrant AI community dedicated to building the future of machine learning. It serves as a collaborative platform where machine learning practitioners can create, discover, and share models, datasets, and AI applications. Hugging Face stands at the intersection of innovation and open collaboration, empowering thousands across the globe to accelerate their AI projects.

## Platform Highlights
- **Models**: Access over 1 million models spanning various AI modalities including text, image, video, audio, and even 3D. Users can browse trending and recently updated models from leaders in the AI space.
- **Datasets**: Discover and utilize from a collection of over 250,000 datasets, facilitating training and fine-tuning of AI models across numerous domains.
- **Spaces**: A hosting environment to deploy and showcase AI applications (over 400,000 applications available), allowing users to run models live and interact with AI-powered demos.
- **Community & Collaboration**: Hugging Face fosters a strong global machine learning community focused on open source contributions, sharing knowledge, and collaborative innovation.
- **Open Source Stack**: Hugging Face offers a rich set of open source tools that empower faster experimentation and deployment in AI workflows.
- **Portfolio Building**: Users can build and showcase their machine learning profiles by sharing work on the platform.

## Enterprise Solutions
Hugging Face provides tailored Team and Enterprise plans for organizations aiming to scale AI efforts with enhanced security and administrative controls:
- **Enterprise-grade security and access controls**
- **Single Sign-On (SSO) integration** for secure identity management
- **Resource regions selection** for data sovereignty and compliance
- **Audit logs** for comprehensive action tracking within the platform
- Flexible contract and pricing options starting at $20/user/month for Teams

## Company Culture
- Commitment to **open collaboration** and knowledge sharing within the AI community
- Focus on **innovation**, enabling AI practitioners at all skill levels to learn, build, and contribute
- Strong emphasis on **inclusivity** through open source, empowering a global audience
- Driven to accelerate AI adoption by making powerful tools and datasets accessible and easy to use

## Customers and Users
- Researchers and developers in academia and industry
- AI startups and technology enterprises leveraging AI to build novel applications
- Data scientists and machine learning engineers seeking to streamline model development and deployment
- Enterprises requiring secure, scalable AI infrastructure and governance capabilities

## Careers & Opportunities
Hugging Face is an exciting place for AI talent who are passionate about open source, community, and cutting-edge machine learning technologies. They offer career opportunities in engineering, research, product, and customer success to help grow the future of AI. Join a mission-driven team dedicated to democratizing AI and empowering creators worldwide.

---

**Join Hugging Face Today**

- Explore 1M+ models and 250K+ datasets  
- Build and share your machine learning portfolio  
- Collaborate with a global community of AI experts  
- Scale your AI solutions with enterprise-grade tools

Visit [huggingface.co](https://huggingface.co) to get started.  
Together, let's build the future of AI.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>